<a href="https://colab.research.google.com/github/Oleonn/DataMining/blob/main/Data_mining_INaturalist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data mining

##Setup and connection to Google Drive

In [2]:
!pip install pyinaturalist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.2 MB/s eta 0:00:00


In [3]:
from pyinaturalist.v1.observations import get_observation_species_counts
import json
from pyinaturalist import (
    Observation,
    pprint,
    get_observations,
    get_observation_species_counts
)

from PIL import Image

import pandas as pd
pd.options.display.max_colwidth = 100  #Sert a augmenter la qte de caracteres affiches pour chaque string
import random

import csv

import requests

import os

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##Directory and parameters

In [28]:
directory = "/content/drive/MyDrive/Projet_mellifere/Donnees/"
sp_classes = ["Asclepias_syriaca", "Daucus_carota", "Eutrochium_maculatum", "Leucanthemum_vulgare", "Solidago_canadensis"]
originals = "originals"

#Create class folders if they don't already exist in directory
for name in sp_classes:
  #Create the species folder
  sp_classes_path = os.path.join(directory, name)
  if not os.path.exists(sp_classes_path):
    os.makedirs(sp_classes_path)
    print(f"folder {sp_classes_path} created.")
  #Create the originals subfolder
  sp_classes_originals = os.path.join(sp_classes_path, originals)
  if not os.path.exists(sp_classes_originals):
    os.makedirs(sp_classes_originals)
    print(f"folder {sp_classes_originals} created.")

folder /content/drive/MyDrive/Projet_mellifere/Donnees/Asclepias_syriaca/originals created.


Before executing the next cell, export the occurences from the GBIF website for the targeted species. Make sure to select "INaturalist (research)" in the search filters. Download the occurences in Archive Darwin Core format. Open the zip file(s) and extract the "multimedia.txt" of every target species into their matching folder (in the species folder that is, not in the "originals" one).

##Data mining

In [ ]:
wished_format = "small" #either thumb, small, medium, original or large"
wished_nb = 2000
originals = "originals"

for name in sp_classes:
  sp_classes_path = os.path.join(directory, name)
  img_list_complete = pd.read_csv(sp_classes_path+"/multimedia.txt", dtype=str, sep="\t")["identifier"].tolist()
  img_list_used = random.sample(img_list_complete, wished_nb)
  for line in range(len(img_list_used)):
    img_list_used[line] = img_list_used[line].replace("original", wished_format)

  sp_classes_originals = os.path.join(sp_classes_path, originals)
  broken_images = []
  count = 1
  for img in img_list_used:
    # We can split the file based on '/' and extract the last split within the Python list below:
    file_name = img.split('/')[-2]
    file_name = f"{sp_classes_originals}/{file_name}.jpeg"  # Update file extension to .jpeg
    if count == 1:
      print(f"Download of {wished_nb} files for {name} has started in {sp_classes_originals}")
    if count % 100 == 0 and count < wished_nb:
      print(f"File {count} out of {wished_nb} for {name} has been downloaded in {sp_classes_originals}")
    if count == wished_nb:
      print(f"All {wished_nb} files for {name} have been downloaded in {sp_classes_originals}")
    count = count + 1
    # Now let's send a request to the image URL:
    r = requests.get(img, stream=True)
    # We can check that the status code is 200 before doing anything else:
    if r.status_code == 200:
        # This command below will allow us to write the data to a file as binary:
        with open(file_name, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)
    else:
        # We will write all of the images back to the broken_images list:
        broken_images.append(img)

  with open(f"{sp_classes_originals}/img_list_used_{name}.csv", "w", newline="") as f:
    for item in img_list_used:
      f.write(item + ",\n")
    print(f"Mission {name} complete.")

print("Mission complete. Good work, 007")